<a id='top'></a>
# A Quantum Approach for Set Cover #

## Table of Contents
1. [Introduction](#set_introduction)
2. [Weighted Set Cover Problem](#set_cover_problem)
3. [Classical Solution using OR-Tools](#classical_solution_OR_Tools)
4. [Conversion of Set Cover to the Quantum Regime](#set_cover_quantum_regime)
5. [Setting up the Problem Using the QC Ware Client API](#qcware_client_API)
    <br>5.a. [Execution via IBM's Qiskit using the QC Ware API](#qcware_API_IBM_qiskit)
    <br>5.b. [Execution via a brute-force simulator](#brute_force_simulator)
    <br>5.c. [Execution via D-Wave's Systems Quantum Annealer using the QC Ware API](#quantum_annealer_API_dwave)
  

## 1. Introduction <a id="set_introduction"></a>
Our next example is a solver for the set cover problem. Given a set of elements $\mathcal{\{1,2,...,n\}}$ (called the universe) and a collection $\mathcal{S}$ of $\mathcal{m}$ sets whose union equals the universe, the set cover problem is to identify the smallest sub-collection of $\mathcal{S}$ whose union equals the universe. For example, consider the universe $\mathcal{U=\{1,2,3,4,5\}}$ and the collection of sets $\mathcal{S=\{\{1,2,3\},\{2,4\},\{3,4\},\{4,5\}\}}$. Clearly the union of $\mathcal{S}$ is $\mathcal{U}$. However, we can cover all of the elements with the following, smaller number of sets: $\mathcal{S=\{\{1,2,3\}, \{4,5\}\}}$[<sup>1</sup>](https://en.wikipedia.org/wiki/Set_cover_problem).

Let's say you run a top secret lab and you'd like to hire a group of security personnel that will watch that lab for 24 hours each day. But you're also on a budget so you don't want to hire more people than you have to. You get a list of candidates and each candidate tells you what times they're available for. Your job is to find the smallest number of candidates such that all hours of the day are accounted for, which is in essence, a set cover problem.

Maybe we have five candidates and there are four 6 hour time slots we're trying to fill. A visual representation of the candidates and their availability might look like this:

![](https://files.slack.com/files-pri/T24940PQV-FDD3C4RT3/example_table.png?pub_secret=908ab1bfbb)

We can see that there are a couple of ways that we could hire candidates to fill up the available time slots. For instance, hiring A, C, and D works, but so does hiring B and E. Since the second option requires hiring only two candidates rather than three, it satisfies the conditions of the set cover problem by giving us the minimum number of candidates required to fill our time space.

***
# 2. Weighted Set Cover Problem <a id="set_cover_problem"></a>

The problem that we describe above is an example of a **weighted set cover** problem. This is a classic problem in combinatorial optimization. 

We can formulate the problem as an integer (binary) optimization problem. Assign a binary variable $x_t \in \{0,1\}$ for every element $t \in \mathcal{T}$, which will be referred to as **subset indicator variables**. Also for all $t \in \mathcal{T}$, and $s \in \mathcal{S}$, we define $c_{ts} = 1$ if $s \in t$, and $c_{ts} = 0$ if $s \notin t$. Then our weighted set cover problem goals can be expressed by the following MILP (Mixed-Integer Linear Program):

$$
\begin{equation}
\begin{split}
\text{minimize} \;\; & \sum_{t \in \mathcal{T}} w(t) \; x_t  \\
\text{subject to} \;\; & \sum_{t \in \mathcal{T}} c_{ts} x_t \geq 1, \;\; \forall \;\; s \in \mathcal{S}  \\
& x_t \in \{0,1\}, \;\; \forall \;\; t \in \mathcal{T}.
\end{split}
\end{equation}
$$

The first constraint expresses the fact that each element $s \in \mathcal{S}$ must be covered by at least one element $t \in \mathcal{T}$, which is the **set cover** constraint, from which the problem derives its name.

***
Now let's apply this to our example above. Suppose $\mathcal{S}$ = time_slots = $\mathcal\{1,2,3,4\}$, and let $\mathcal{T}$ = candidates = $\mathcal\{a,b,c,d,e\}$, where 

$$
\begin{equation}
\begin{split}
a &= \{1\} \\
b &= \{1,3\} \\
c &= \{3,4\} \\
d &= \{2\} \\
e &= \{2,4\}
\end{split}
\end{equation}
$$

We will represent $c_{ts}$ using a cost matrix $C$ defined below, with rows representing each of the 5 candidates, and columns representing their availability for each time slot (1 for available, 0 for not available)

$$
C = 
\begin{bmatrix}
1 & 0 & 0 & 0 \\
1 & 0 & 1 & 0 \\
0 & 0 & 1 & 1 \\
0 & 1 & 0 & 0 \\
0 & 1 & 0 & 1
\end{bmatrix}
\;\;
$$

We also need to define a cost function $w$, which will tell us how much it costs to employ an employee for a single shift. To simplify things, we'll just say that all shifts have a cost of 1: 

$$w(t) = 1$$ 

Thus, we're trying to pick the combination of candidates that minimizes the number of shifts we have to pay for while still covering all shifts available.

***
## 3. Classical solution using ```OR-Tools```<a id="classical_solution_OR_Tools"></a>
```OR-Tools``` is an open source software suite for optimization available from Google, which can be used with both commercial and open-source solvers. In this example we will use Google's ```GLOP``` solver which comes standard with the ```OR-Tools``` installation. More information on ```OR-Tools``` can be found at the <a href="https://developers.google.com/optimization/introduction/overview" style="text-decoration: none;">OR-Tools homepage</a>. The user guide can be found <a href="https://jupyter-notebook.readthedocs.io/en/stable/notebook.html" style="text-decoration: none;">here</a>, which contains extensive documentation and lots of examples.

We now go through the step by step solution for the set cover problem in Python, using ```ortools```.

We first import the solver library from ```ortools```.

In [1]:
from qcware import forge
# this line is for internal tracking; it is not necessary for use!
forge.config.set_environment_source_file('set_cover.ipynb')

#NBVAL_IGNORE_OUTPUT
!pip install ortools
from ortools.linear_solver import pywraplp

We now represent the problem data and print it for the user.

In [2]:
# Represent the problem data
time_slots = [1, 2, 3, 4]
candidates = {'a':{1}, 'b':{1, 3}, 'c':{3, 4}, 'd':{2}, 'e':{2,4}}

# Print the problem
print("The time slots are:")
for item in time_slots:
    print(item)

print("\n")
print("The candidates' available times:")
for key, val in candidates.items():
    print(key, ":", val)

The time slots are:
1
2
3
4


The candidates' available times:
a : {1}
b : {1, 3}
c : {3, 4}
d : {2}
e : {2, 4}


Next we instantiate the classical solver, and obtain the ```solver``` object.

In [3]:
# Instantiate a mixed-integer solver, naming it Weighted-Set-Cover
solver = pywraplp.Solver('Weighted-Set-Cover', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

First, we define the binary problem variables.

In [4]:
# Define integer binary variables.
xa = solver.IntVar(0, 1, 'a')
xb = solver.IntVar(0, 1, 'b')
xc = solver.IntVar(0, 1, 'c')
xd = solver.IntVar(0, 1, 'd')
xe = solver.IntVar(0, 1, 'e')

Next, we set the constraints of the problem. To get the constraints, we simply read down each column of the cost matrix $C$. For instance, column 1 represents the first time slot. Since candidates A and B are the only ones available at this time, our first constraint is that we must hire either candidate A or B (or both).

In [5]:
# Constraint 1: xa + xb >= 1
constraint1 = solver.Constraint(1, solver.infinity())
constraint1.SetCoefficient(xa, 1)
constraint1.SetCoefficient(xb, 1)

# Constraint 2: xd + xe >= 1
constraint2 = solver.Constraint(1, solver.infinity())
constraint2.SetCoefficient(xd, 1)
constraint2.SetCoefficient(xe, 1)

# Constraint 3: xb + xc >= 1
constraint3 = solver.Constraint(1, solver.infinity())
constraint3.SetCoefficient(xb, 1)
constraint3.SetCoefficient(xc, 1)

# Constraint 4: xc + xe >= 1
constraint4 = solver.Constraint(1, solver.infinity())
constraint4.SetCoefficient(xc, 1)
constraint4.SetCoefficient(xe, 1)

The objective function for the problem is then set.

In [6]:
# Minimize xa + xb + xc + xd + xe
objective = solver.Objective()
objective.SetCoefficient(xa, 1)
objective.SetCoefficient(xb, 1)
objective.SetCoefficient(xc, 1)
objective.SetCoefficient(xd, 1)
objective.SetCoefficient(xe, 1)
objective.SetMinimization()

We finally solve the problem and then verify that an optimal solution has been found.

In [7]:
# Solve the problem and print the solution
result_status = solver.Solve()

# Verify the problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

Let's print the solution to confirm the result.

In [8]:
# Print the selected sets in the optimal solution
print("\n")
print("The best candidates are:")
for item in ['a', 'b', 'c', 'd', 'e']:
    var = solver.LookupVariable(item)
    if var.solution_value() == 1:
        print(item, ": ", candidates[item])



The best candidates are:
b :  {1, 3}
e :  {2, 4}


***
## 4. Conversion of Set Cover to the Quantum Regime <a id="set_cover_quantum_regime"></a>
We're now ready to look at how to solve this problem in the quantum regime. We first need to convert our problem into a quadratic unconstrained binary optimazation, or QUBO, problem. One advantage to this approach is that it makes the problem ammenable to both existing annealing and gate-based quantum computing hardware architectures.

For Operations Research professionals wondering how to formulate their problems as a QUBO, this paper <a href="https://arxiv.org/pdf/1302.5843.pdf" style="text-decoration: none;">Ising formulations of many NP problems</a>, by Andrew Lucas, 2014, gives Ising formulations of many NP problems, which one can in turn use to obtain the QUBO forms suitable for multiple backend solvers. In particular, the paper also covers the **set cover** problem. We will follow the same method outlined in this paper.

**Note: There are many ways to convert the set cover problem to QUBO form. Here we focus on a particular reduction.**

We recall the set cover problem expressed as an optimization problem

$$
\begin{equation}
\begin{split}
\text{minimize} \;\; & \sum_{t \in \mathcal{T}} w(t) \; x_t  \\
\text{subject to} \;\; & \sum_{t \in \mathcal{T}} c_{ts} x_t \geq 1, \;\; \forall \;\; s \in \mathcal{S}  \\
& x_t \in \{0,1\}, \;\; \forall \;\; t \in \mathcal{T}.
\end{split}
\end{equation}
$$

The basic idea of the reduction in the <a href="https://arxiv.org/pdf/1302.5843.pdf" style="text-decoration: none;">Lucas' paper</a> is to introduce additional binary variables into the problem, and find an equivalent problem where the system of inequality constraints become equality constraints. To this extent we introduce the binary variables $y_{s,m}$ which serve as a count of the number of times each element of $\mathcal{S}$ has been covered. Specifically

$$
y_{s,m} =
\begin{cases}
1 & \;\; \text{if} \;\; s \in \mathcal{S} \text{ is covered } m \text{ times} \\
0 & \;\; \text{otherwise}.
\end{cases}
$$

The variables $y_{s,m}$ will be called **ancilla binary variables**. We want each element to be covered at least once, and thus we need $m \geq 1$. However we can also easily find an upper bound on $m$, namely the maximum number of times any element will be covered if we included all the elements $t \in \mathcal{T}$ in our solution. Let this number be $M$.

Now for every element $s \in \mathcal{S}$, exactly one of the variables $y_{s,m}$ is $1$, and all the rest are $0$, for all $1 \leq m \leq M$, and for fixed $s$, and thus we have the following constraints

$$
\begin{split}
& \sum_{m=1}^{M} y_{s,m} = 1 \\
& \sum_{m=1}^{M} m y_{s,m} = \sum_{t \in \mathcal{T}} c_{ts} x_t,
\end{split}
$$

for all $s \in \mathcal{S}$.

We can thus restate the goals of the weighted set cover problem as the following equivalent binary optimization problem:

$$
\begin{equation}
\begin{split}
\text{minimize} \;\; & \sum_{t \in \mathcal{T}} w(t) \; x_t \\
\text{subject to} \;\; & \sum_{m=1}^{M} y_{s,m} = 1, \;\; \forall \;\; s \in \mathcal{S} \\
& \sum_{m=1}^{M} m y_{s,m} = \sum_{t \in \mathcal{T}} c_{ts} x_t, \;\; \forall \;\; s \in \mathcal{S} \\
& x_t \in \{0,1\}, \;\; \forall \;\; t \in \mathcal{T}.
\end{split}
\end{equation}
$$

The final step involves conversion of the above optimization problem into QUBO form, by including the equality constraints into the objective function as a penalty term after squaring and summing all of them, leading to the following problem:

$$
\begin{equation}
\begin{split}
\text{minimize} \;\; & H_A + H_B  \\
\text{subject to} \;\; & x_t \in \{0,1\}, \;\; \forall \;\; t \in \mathcal{T},
\end{split}
\end{equation}
$$

where

$$
\begin{equation}
\begin{split}
H_A &= A \sum_{s \in \mathcal{S}} \left( \sum_{m=1}^{M} y_{s,m} - 1 \right)^2 + A \sum_{s \in \mathcal{S}} \left( \sum_{m=1}^{M} m y_{s,m} - \sum_{t \in \mathcal{T}} c_{ts} x_t \right)^2 \\
H_B &= \sum_{t \in \mathcal{T}} w(t) \; x_t \;,
\end{split}
\end{equation}
$$
and $A$ is a sufficiently large constant.

The problem we obtained reduces to the formulation of set cover in the <a href="https://arxiv.org/pdf/1302.5843.pdf" style="text-decoration: none;">Lucas' paper</a>, for the special case $w(t) = 1$, for all $t \in \mathcal{T}$.

Using QC Ware's API, we can solve this problem with the `optimize_binary()` function.  With the `optimize_binary()` function, the user has the option to choose from a number of different backend solvers.  Without having to change the way the QUBO is input, QC Ware's tools then automatically formulate the problem in a way that is suitable for the selected solver's corresponding software and hardware environment.  For this particular example, we will demonstrate how to use two solvers on the QC Ware API. The first is `qiskit_aqua_qaoa`, which is an implementation of the QAOA algorithm [<sup>2</sup>](https://arxiv.org/abs/1411.4028) for (classical) energy minimization problems of the Ising model, developed using IBM's Qiskit [<sup>3</sup>](https://qiskit.org) Aqua [<sup>4</sup>](https://qiskit.org/aqua) software framework. The second solver is the D-Wave quantum annealing-based hardware.

***
## 5. Setting up the Problem Using the QC Ware Client API <a id="qcware_client_API"></a>
We first have to import the QC Ware module and enter your API key (you can find your API key on your dashboard on [Forge](https://forge.qcware.com))

We'll also import python's pretty-printing module `pprint` for a few displays.

In [9]:
import qcware.types
import pprint

We'll then define the various time slots we're looking to fill by representing them sequentially as time-slot 1, 2, 3, or 4:

In [10]:
time_slots = {1, 2, 3, 4}

We define our candidates a dictionary mapping the candidate name to the time slots that they can cover.

In [11]:
candidates = {
    'cA': {1}, 
    'cB': {1, 3}, 
    'cC': {3, 4}, 
    'cD': {2}, 
    'cE': {2, 4}
}

Because of the way that we have to formulate the problem as a QUBO, we also have to define a variable `B` as the cost of having a set. In our example, we can this of this as how many salaries we have to pay for each candidate that we hire. So, we set B = 1 since each person we hire will cost us one salary and our job will be to minimize the number of salaries we have to pay. We'll also define a variable `A`, which is an arbitrary coefficient necessary for formulating the problem and is equivalent to the "big-M" parameter in mathematical modeling.

In [12]:
B = 1.0
A = 3.0

We now have to generate a Q matrix that fits the QUBO formulation that we defined earlier. For a more detailed description of the format expected for the Q matrix by the QC Ware API, please see the Getting Started Guide.

We next want to initialize the Q dictionary. Keys of Q will be represented by tuples (i,j), which correspond to an (i,j) position in the Q-matrix. The values within Q are the matrix element in the (i,j) position of the Q-matrix. Will use the Python package `qubovert` to deal with our QUBO manipulations. We will create a Polynomial Constrained Boolean Optimization (PCBO) object. 

First we create our boolean variables $x_t$.

In [13]:
# !pip install qubovert
from qubovert import boolean_var

x = {c: boolean_var(c) for c in candidates}

Next we create the objective function that we want to minimize, $$H = B\sum_{t \in \mathcal{T}} x_t.$$

In [14]:
H = B * sum(x.values())
pprint.pprint(H)

{('cA',): 1.0, ('cB',): 1.0, ('cC',): 1.0, ('cD',): 1.0, ('cE',): 1.0}


Next we add the constraint that 
$$\forall \;\; s \in \mathcal{S} \quad \sum_{t \in \mathcal{T}} c_{ts} x_t \geq 1,$$
or equivalently,
$$\forall \;\; s \in \mathcal{S} \quad \sum_{t \in \mathcal{T}} c_{ts} x_t - 1\geq 0.$$
We enforce this constraint with the lagrange multiplier $\lambda = A$. 

In [15]:
for t in time_slots:
    H.add_constraint_ge_zero(
        sum(x[c] for c, v in candidates.items() if t in v) - 1,
        lam=A
    )
    
pprint.pprint(H)

{(): 12.0,
 ('cA',): -2.0,
 ('cB',): -5.0,
 ('cB', 'cA'): 3.0,
 ('cB', 'cC'): 3.0,
 ('cC',): -5.0,
 ('cD',): -2.0,
 ('cE',): -5.0,
 ('cE', 'cC'): 3.0,
 ('cE', 'cD'): 3.0}


Notice that ancilla variables were added to enforce the inequality constraints., labeled by `'__a'`.

We note that the degree of `H` is only 2 and is therefore already a QUBO.

In [16]:
H.degree

2

We will still do the standard procedure since it works even if the degree of `H` is greater than 2.

In [17]:
qubo = H.to_qubo()
pprint.pprint(qubo)

{(): 12.0,
 (0,): -2.0,
 (0, 1): 3.0,
 (1,): -5.0,
 (1, 2): 3.0,
 (2,): -5.0,
 (2, 4): 3.0,
 (3,): -2.0,
 (3, 4): 3.0,
 (4,): -5.0}


### 5.a. Execution via IBM's Qiskit using the QC Ware API<a id="qcware_API_IBM_qiskit"></a>

We're now ready to solve the problem using QC Ware's API on the IBM simulator or IBM 20 qubit chip. The following call takes as arguments the BinaryProblem class for any QUBO problem and an API key necessary for authentication, embeds the QUBO problem on IBM's QAOA implementation, initiates the execution, and returns the solution found.

At the present time, QAOA simulation takes a long time (upwards of an hour!) so this is commented out; uncomment it if you have some time to spare!

In [18]:
#problem=qcware.types.optimization.BinaryProblem()
#problem=problem.set_problem(qubo)

#solution = qcware.optimization.optimize_binary(instance=problem, key=QCWARE_API_KEY, solver='ibm_sw_qaoa')

### 5.b. Execution via a brute-force simulator<a id="brute_force_simulator"></a>

For very small problems, a brute-force approach iterates through every possibility and finds the most optimal result.  For obvious reasons, this is a *terrible* approach for a problem of real size, but is invaluable for troubleshooting and verifying small problems.

In [19]:
poly = qcware.types.optimization.PolynomialObjective(
    polynomial=qubo.Q,
    num_variables=qubo.num_binary_variables,
    domain='boolean'
    )
problem = qcware.types.optimization.BinaryProblem(objective=poly)

solution = forge.optimization.optimize_binary(instance=problem, backend='qcware/cpu')
print(solution)

Objective value: -10
Solution: (0, 1, 0, 0, 1)


With this approach to Set Cover, we require up to $N+n(1+\log N)$ logical variables (recall that $N$ is the number of time slots, and $n$ is the number of candidates).  This means, for instance, that on IBM's $20$-qubit chip, we can in principle run problem sizes where $N+n(1+\log N)\leq20$.

Lets look at the solution.

Notice that we use the `H.convert_solution` method to convert the solution to the QUBO back to the solution to the PCBO. Notice also that we use the `H.remove_ancilla_from_solution` method to remove the ancilla information that was just used to enforce the inequality constraints. Finally, the `H.is_solution_valid` method checks to see if all the constraints are satisfied.

In [20]:
lowest_value_bitstring = solution.lowest_value_bitstring
H_solution = H.convert_solution(lowest_value_bitstring)
print("solution:", H.remove_ancilla_from_solution(H_solution))

print("solution is", "valid" if H.is_solution_valid(H_solution) else "invalid")

solution: {'cA': 0, 'cB': 1, 'cC': 0, 'cD': 0, 'cE': 1}
solution is valid


We see that the best solution is the one where candidate B and candidate E cover their available times.

### 5.c. Execution via D-Wave's Systems Quantum Annealer using the QC Ware API<a id="quantum_annealer_API_dwave"></a>

Alternatively, we can solve this problem using the same setup and QC Ware's API on the D-Wave Systems Quantum Annealer[<sup>2</sup>](https://www.dwavesys.com/tutorials/background-reading-series/introduction-d-wave-quantum-hardware).

The following call takes as arguments the `BinaryProblem` class for any QUBO problem and an API key necessary for authentication, embeds the QUBO problem on the quantum annealing hardware, initiates the quantum annealing process, and returns the best solution found by that process.

In [21]:
poly = qcware.types.optimization.PolynomialObjective(
    polynomial=qubo.Q,
    num_variables=qubo.num_binary_variables,
    domain='boolean'
    )
problem = qcware.types.optimization.BinaryProblem(objective=poly)

solution = forge.optimization.optimize_binary(instance=problem, backend='dwave/2000q')
print(solution)

Objective value: -10
Solution: (0, 1, 0, 0, 1)


The round trip time for this quantum computation is approximately 5 seconds and is independent of the size of the problem. With the default parameters during the "optimize_binary" call the quantum annealer actually runs the problem 10,000 times, each one taking approximately 5 micro-seconds. We then select the best solution found by the annealer across all these runs and return that back to the user. 

With this approach to Set Cover, we require up to $N+n(1+\log N)$ (we briefly recall again that $N$ is the number of time slots, and $n$ is the number of candidates) logical variables.  This means that on an D-Wave 2000Q, we can in principle run problem sizes where $N+n(1+\log N)\leq60$.

We can display the solution just as we did above:

In [22]:
H_solution = H.convert_solution(solution.lowest_value_bitstring)
print("solution:", H.remove_ancilla_from_solution(H_solution))

print("solution is", "valid" if H.is_solution_valid(H_solution) else "invalid")

solution: {'cA': 0, 'cB': 1, 'cC': 0, 'cD': 0, 'cE': 1}
solution is valid


Looking back at the figure from earlier, we can easily verify the correctness of the solution. However, it is not necessarily optimal (whereas brute-force always is).

![](https://files.slack.com/files-pri/T24940PQV-FDD3C4RT3/example_table.png?pub_secret=908ab1bfbb)

Congratulations, you just used a quantum circuit-based computer and a quantum annealer to solve an NP-complete problem!

Note: If you are viewing this on GitHub and want to access QC Ware's API to execute the notebook you will need an API key. Please reach out support@qcware.com to request access.
<br><a href="#top">Back to Table of Contents</a>